In [ ]:
!git config --global user.email "mm5523@ic.ac.uk"
!git config --global user.name "mau-mar"

In [ ]:
cd /content/drive/MyDrive/AppliedMachineLearning/AdvancedDeepLearningSystems/MASErepo2/mase2/machop

In [ ]:
!chmod 755 -R "/content/drive/MyDrive/AppliedMachineLearning/AdvancedDeepLearningSystems/MASErepo2/mase2/machop"

In [ ]:
!python -m pip install -r requirements.txt


In [ ]:
!./ch train jsc-tiny jsc --max-epochs 10 --batch-size 256 # BASELINE jsc-tiny

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/drive/MyDrive/AppliedMachineLearning/AdvancedDeepLearningSystems/MASErepo2/mase2/mase_output/jsc_tiny_lab1_experiments/learning_rate_experiments/software/tensorboard/lightning_logs


In [ ]:
# Try model 10x larger than toy network

!./ch train jsc-less-tiny jsc --max-epochs 10 --batch-size 256

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/drive/MyDrive/AppliedMachineLearning/AdvancedDeepLearningSystems/MASErepo2/mase2/mase_output/jsc_tinyX10/software/tensorboard/lightning_logs


___

# Lab 2

In [ ]:
import sys
import logging
import os
from pathlib import Path
from pprint import pprint as pp

from chop.dataset import MaseDataModule, get_dataset_info
from chop.tools.logger import set_logging_verbosity

from chop.passes.graph import (
    save_node_meta_param_interface_pass,
    report_node_meta_param_analysis_pass,
    profile_statistics_analysis_pass,
    add_common_metadata_analysis_pass,
    init_metadata_analysis_pass,
    add_software_metadata_analysis_pass,
)
from chop.tools.get_input import InputGenerator
from chop.tools.checkpoint_load import load_model
from chop.ir import MaseGraph

from chop.models import get_model_info, get_model

set_logging_verbosity("info")

In [ ]:
# Set Up the Dataset
batch_size = 8
model_name = "jsc-tiny"
dataset_name = "jsc"


data_module = MaseDataModule(
    name=dataset_name,
    batch_size=batch_size,
    model_name=model_name,
    num_workers=0,
)
data_module.prepare_data()
data_module.setup()

In [ ]:
# Set Up the Model
CHECKPOINT_PATH = "/content/drive/MyDrive/AppliedMachineLearning/AdvancedDeepLearningSystems/MASErepo2/mase2/mase_output/jsc_tiny_baseline/software/training_ckpts/best-jsc_tiny_ep10_bs256_baseline.ckpt"
model_info = get_model_info(model_name)
model = get_model(
    model_name,
    task="cls",
    dataset_info=data_module.dataset_info,
    pretrained=False)

model = load_model(load_name=CHECKPOINT_PATH, load_type="pl", model=model)

In [ ]:
# Get Dummy Data In

# get the input generator
input_generator = InputGenerator(
    data_module=data_module,
    model_info=model_info,
    task="cls",
    which_dataloader="train",
)

# a demonstration of how to feed an input value to the model
dummy_in = next(iter(input_generator))
_ = model(**dummy_in)

In [ ]:
# generate the mase graph and initialize node metadata
mg = MaseGraph(model=model)

In [ ]:
# Analysis pass: it does not change the graph

mg, _ = init_metadata_analysis_pass(mg, None)
mg, _ = add_common_metadata_analysis_pass(mg, {"dummy_in": dummy_in})
mg, _ = add_software_metadata_analysis_pass(mg, None)

In [ ]:
# report graph is an analysis pass that shows you the detailed information in the graph
from chop.passes.graph import report_graph_analysis_pass
_ = report_graph_analysis_pass(mg)

In [ ]:
# Profile Statistics Pass
pass_args = {
    "by": "type",                                                            # collect statistics by node name
    "target_weight_nodes": ["linear"],                                       # collect weight statistics for linear layers
    "target_activation_nodes": ["relu"],                                     # collect activation statistics for relu layers
    "weight_statistics": {
        "variance_precise": {"device": "cpu", "dims": "all"},                # collect precise variance of the weight
    },
    "activation_statistics": {
        "range_quantile": {"device": "cpu", "dims": "all", "quantile": 0.97} # collect 97% quantile of the activation range
    },
    "input_generator": input_generator,                                      # the input generator for feeding data to the model
    "num_samples": 32,                                                       # feed 32 samples to the model
}

mg, _ = profile_statistics_analysis_pass(mg, pass_args)
mg, _ = report_node_meta_param_analysis_pass(mg, {"which": ("software",)})

In [ ]:
# Transform Pass: Quantization

pass_args = {
    "by": "type",
    "default": {"config": {"name": None}},
    "linear": {
        "config": {
            "name": "integer",
            # data
            "data_in_width": 8,
            "data_in_frac_width": 4,
            # weight
            "weight_width": 8,
            "weight_frac_width": 4,
            # bias
            "bias_width": 8,
            "bias_frac_width": 4,
        }
    },
}

from chop.passes.graph.transforms import (
    quantize_transform_pass,
    summarize_quantization_analysis_pass,
)
from chop.ir.graph.mase_graph import MaseGraph


ori_mg = MaseGraph(model=model)
ori_mg, _ = init_metadata_analysis_pass(ori_mg, None)
ori_mg, _ = add_common_metadata_analysis_pass(ori_mg, {"dummy_in": dummy_in})

mg, _ = quantize_transform_pass(mg, pass_args)
summarize_quantization_analysis_pass(ori_mg, mg, save_dir="quantize_summary")

In [ ]:
# Lab 2 Question 4


# from chop.passes.graph.utils

def get_mase_op(node):
    return node.meta["mase"].parameters["common"]["mase_op"]

def get_mase_type(node):
    return node.meta["mase"].parameters["common"]["mase_type"]

def get_node_actual_target(node):
  """
  return the actual target of the node
  - for "call_module": return the torch.nn.Module instance
  - for "call_function": return the function
  - for others: return the node.target
  """
  if node.op == "call_module":
      return node.meta["mase"].module
  elif node.op == "call_function":
      return node.target
  else:
      return node.target

def compare_graph_nodes(ori_mg, mg):
  # Traverse the graph
  for transformed_mg_node, original_ms_node in zip(mg.fx_graph.nodes, ori_mg.fx_graph.nodes):

    # Check if the types of the actual targets of the nodes are different
    if (type(get_node_actual_target(transformed_mg_node)) != type(get_node_actual_target(original_ms_node))):

        # Get the types of the original and transformed nodes
        original_node_type = type(get_node_actual_target(original_ms_node))
        quantized_node_type = type(get_node_actual_target(transformed_mg_node))

        # Same for both original and transformed MaseGraph
        print(f' Node Name: {transformed_mg_node.name}')
        print(f' Node Mase Type: {get_mase_type(transformed_mg_node)}')
        print(f' Node Mase Operation: {get_mase_op(transformed_mg_node)}')
        # Module difference
        print(f' Original Module: {original_node_type}')
        print(f' New Module: {quantized_node_type}\n')

# Example usage
compare_graph_nodes(ori_mg, mg)


In [ ]:
# Lab 2 Question 5

model_name = 'jsc-less-tiny'

# Set Up the Model
CHECKPOINT_PATH = "/content/drive/MyDrive/AppliedMachineLearning/AdvancedDeepLearningSystems/MASErepo2/mase2/mase_output/jsc_tinyX10/software/training_ckpts/best_ep10_bs256.ckpt"
model_info = get_model_info(model_name)
model = get_model(
    model_name,
    task="cls",
    dataset_info=data_module.dataset_info,
    pretrained=False)

model = load_model(load_name=CHECKPOINT_PATH, load_type="pl", model=model)

# Get Dummy Data In
# get the input generator
input_generator = InputGenerator(
    data_module=data_module,
    model_info=model_info,
    task="cls",
    which_dataloader="train",
)

# a demonstration of how to feed an input value to the model
dummy_in = next(iter(input_generator))
_ = model(**dummy_in)

# generate the mase graph and initialize node metadata
mg = MaseGraph(model=model)

# Analysis pass: it does not change the graph

mg, _ = init_metadata_analysis_pass(mg, None)
mg, _ = add_common_metadata_analysis_pass(mg, {"dummy_in": dummy_in})
mg, _ = add_software_metadata_analysis_pass(mg, None)


# Transform Pass: Quantization

pass_args = {
    "by": "type",
    "default": {"config": {"name": None}},
    "linear": {
        "config": {
            "name": "integer",
            # data
            "data_in_width": 8,
            "data_in_frac_width": 4,
            # weight
            "weight_width": 8,
            "weight_frac_width": 4,
            # bias
            "bias_width": 8,
            "bias_frac_width": 4,
        }
    },
}

from chop.passes.graph.transforms import (
    quantize_transform_pass,
    summarize_quantization_analysis_pass,
)
from chop.ir.graph.mase_graph import MaseGraph


ori_mg = MaseGraph(model=model)
ori_mg, _ = init_metadata_analysis_pass(ori_mg, None)
ori_mg, _ = add_common_metadata_analysis_pass(ori_mg, {"dummy_in": dummy_in})

mg, _ = quantize_transform_pass(mg, pass_args)
summarize_quantization_analysis_pass(ori_mg, mg, save_dir="quantize_summary")

In [ ]:
# Lab 2 Question 6

from tabulate import tabulate

# Initialize an empty list to store node comparison data
comparison_data = []
comparison_weight = []
comparison_bias = []

for node_ori, node_quant in zip(ori_mg.fx_graph.nodes, mg.fx_graph.nodes):
    # Extracting original and quantized model information
    try:
      row_data = [
          node_ori.meta['mase'].node,
          node_ori.meta['mase'].parameters['common']['mase_op'],
          node_ori.meta['mase'].parameters['common']['args']['data_in_0'].get('type', 'N/A'),
          node_ori.meta['mase'].parameters['common']['args']['data_in_0'].get('precision', 'N/A'),
          node_quant.meta['mase'].parameters['common']['args']['data_in_0'].get('type', 'N/A'),
          node_quant.meta['mase'].parameters['common']['args']['data_in_0'].get('precision', 'N/A'),
      ]
      row_weight = [
          node_ori.meta['mase'].node,
          node_ori.meta['mase'].parameters['common']['mase_op'],
          node_ori.meta['mase'].parameters['common']['args']['weight'].get('type', 'N/A'),
          node_ori.meta['mase'].parameters['common']['args']['weight'].get('precision', 'N/A'),
          node_quant.meta['mase'].parameters['common']['args']['weight'].get('type', 'N/A'),
          node_quant.meta['mase'].parameters['common']['args']['weight'].get('precision', 'N/A'),
      ]
      row_bias = [
          node_ori.meta['mase'].node,
          node_ori.meta['mase'].parameters['common']['mase_op'],
          node_ori.meta['mase'].parameters['common']['args']['bias'].get('type', 'N/A'),
          node_ori.meta['mase'].parameters['common']['args']['bias'].get('precision', 'N/A'),
          node_quant.meta['mase'].parameters['common']['args']['bias'].get('type', 'N/A'),
          node_quant.meta['mase'].parameters['common']['args']['bias'].get('precision', 'N/A')
      ]
    except:
      row_data = [
          node_ori.meta['mase'].node,
          node_ori.meta['mase'].parameters['common']['mase_op'],
          "N/A",
          "N/A",
          "N/A",
          "N/A",
      ]
      row_weight = [
          node_ori.meta['mase'].node,
          node_ori.meta['mase'].parameters['common']['mase_op'],
          "N/A",
          "N/A",
          "N/A",
          "N/A",
      ]
      row_bias = [
          node_ori.meta['mase'].node,
          node_ori.meta['mase'].parameters['common']['mase_op'],
          "N/A",
          "N/A",
          "N/A",
          "N/A",
      ]
    comparison_data.append(row_data)
    comparison_weight.append(row_weight)
    comparison_bias.append(row_bias)

headers_data = ["Node", "MASE OP", "Original Data DType", "Original Data Precision", "Quantized Data DType", "Quantized Data Precision"]
headers_weight = ["Node", "MASE OP", "Original Weight DType", "Original Weight Precision", "Quantized Weight DType", "Quantized Weight Precision"]
headers_bias = ["Node", "MASE OP", "Original Bias DType", "Original Bias Precision", "Quantized Bias DType", "Quantized Bias Precision"]

table_data = tabulate(comparison_data, headers=headers_data, tablefmt="grid")
print(table_data)
print("\n")
table_weight = tabulate(comparison_weight, headers=headers_weight, tablefmt="grid")
print(table_weight)
print("\n")
table_bias = tabulate(comparison_bias, headers=headers_bias, tablefmt="grid")
print(table_bias)

In [ ]:
# L2Q6 - continued
import pandas as pd

for ori_node, quant_node in zip(ori_mg.fx_graph.nodes, mg.fx_graph.nodes):
  if type(quant_node.meta['mase'].module).__name__ == "LinearInteger":
    print(f"Original Weights - Layer {quant_node.meta['mase'].module}")
    print(f"\t{quant_node.meta['mase']['common']['args']['weight']}")
    print(f"Quantized Weights - Layer {quant_node.meta['mase'].module}")
    print(f"\t{quant_node.meta['mase'].module.w_quantizer(quant_node.meta['mase'].module.weight)}")


In [ ]:
import matplotlib.pyplot as plt
tensor_original = quant_node.meta['mase']['common']['args']['weight']['value'].detach()
tensor_quantized = quant_node.meta['mase'].module.w_quantizer(quant_node.meta['mase'].module.weight).detach()


flat_tensor_original = tensor_original.flatten()
flat_tensor_quantized = tensor_quantized.flatten()

# Plot histograms
plt.hist(flat_tensor_original.numpy(), bins=50, alpha=0.5, label='Original Weights')
plt.hist(flat_tensor_quantized.numpy(), bins=50, alpha=0.5, label='Quantized Weights')
plt.xlabel('Value', fontsize=16)
plt.ylabel('Frequency', fontsize=16)
plt.legend(loc='upper right')
plt.title('Histogram Comparison: Original vs Quantized Weights')
plt.show()

In [ ]:
# Lab 2 Question 7

!./ch transform --config configs/examples/jsc_less_tiny_by_type_quantize_L2Q7.toml --task cls --accelerator='cpu'

In [ ]:
#___
# Lab 2 - Optional: Write pass to count FLOPs and BitOPs
#___

# from chop.passes.graph.analysis.flop_estimator.calc_modules import calculate_modules

import torch

def get_multipliers(weight=None, bias=None, data=None):
    if weight is not None:
        if weight.dtype == torch.float32:
          weight_multiplier = 32
        elif weight.dtype == torch.float16:
          weight_multiplier = 16
        else:
          weight_multiplier = 8
    else:
        weight_multiplier = 1

    if bias is not None:
        if bias.dtype == torch.float32:
          bias_multiplier = 32
        elif bias.dtype == torch.float16:
          bias_multiplier = 16
        else:
          bias_multiplier = 8
    else:
        bias_multiplier = 1

    if data is not None:
        if data == torch.float32:
          data_multiplier = 32
        elif data == torch.float16:
          data_multiplier = 16
        else:
          data_multiplier = 8
    else:
        data_multiplier = 1

    return {
        "weight_multiplier": weight_multiplier,
        "bias_multiplier": bias_multiplier,
        "data_multiplier": data_multiplier,
    }

def calculate_modules(module, node, in_data, out_data, include_bias=True):
    if isinstance(module, torch.nn.Linear):
        # One computation per weight, for each batch element.

        assert len(in_data) == 1
        batch = in_data[0].numel() / in_data[0].shape[-1]

        multipliers = {
            "weight_multiplier" : node.meta['mase'].parameters['common']['args']['weight']['precision'][0],
            "bias_multiplier" : node.meta['mase'].parameters['common']['args']['bias']['precision'][0],
            "data_multiplier" : node.meta['mase'].parameters['common']['args']['data_in_0']['precision'][0]
        }

        if include_bias:
          #multipliers = get_multipliers(weight=module.weight, bias=module.bias, data=in_data[0])

          computations = ( module.weight.numel() + module.bias.numel() ) * batch
          bitops = ( (module.weight.numel() * multipliers['weight_multiplier']) + (module.bias.numel() * multipliers['bias_multiplier']) ) * batch * multipliers['data_multiplier']
          backward_computations = module.weight.numel() * batch * 2
        else:
          #multipliers = get_multipliers(weight=module.weight, data=in_data[0])

          computations = module.weight.numel() * batch
          bitops = (module.weight.numel() * multipliers['weight_multiplier']) * batch * multipliers['data_multiplier']
          backward_computations = module.weight.numel() * batch * 2

        input_size = in_data[0].numel()
        output_size = out_data[0].numel()
        return {
            "total_parameters": module.weight.numel(),
            "bitops": bitops,
            "computations": computations,
            "backward_computations": backward_computations,
            "input_buffer_size": input_size,
            "output_buffer_size": output_size,
        }

    elif isinstance(module, torch.nn.modules.activation.ReLU) or isinstance(
        module, torch.nn.modules.activation.ReLU6
    ):
        multipliers = get_multipliers(data=in_data[0])
        # ReLU does a single negation check
        return {
            "total_parameters": 0,
            "computations": in_data[0].numel(),
            "bitops": in_data[0].numel() * multipliers['data_multiplier'],
            "backward_computations": in_data[0].numel(),
            "input_buffer_size": in_data[0].numel(),
            "output_buffer_size": out_data[0].numel(),
        }

    elif isinstance(module, torch.nn.LayerNorm):
        multipliers = {
            "data_multiplier" : node.meta['mase'].parameters['common']['args']['data_in_0']['precision'][0]
        }

        #multipliers = get_multipliers(data=in_data[0])

        computations = in_data[0].numel() * 5

        return {
            "total_parameters": 0,
            "computations": computations,
            "bitops": computations * multipliers['data_multiplier'],
            "backward_computations": in_data[0].numel() * 5,
            "input_buffer_size": in_data[0].numel(),
            "output_buffer_size": out_data[0].numel(),
        }

    elif isinstance(module, torch.nn.modules.batchnorm.BatchNorm1d):
        multipliers = {
            "data_multiplier" : node.meta['mase'].parameters['common']['args']['data_in_0']['precision'][0]
        }

        #multipliers = get_multipliers(data=in_data[0])

        # Accesses to E[x] and Var[x] (all channel size)
        total_parameters = 2 * module.num_features
        # (x-running_mean)/running variance
        # multiply by gamma and beta addition
        computations = 4 * in_data[0].numel()
        backward_computations = 4 * in_data[0].numel()

        return {
            "total_parameters": total_parameters,
            "computations": computations,
            "bitops": computations * multipliers['data_multiplier'],
            "backward_computations": backward_computations,
            "input_buffer_size": in_data[0].numel(),
            "output_buffer_size": out_data[0].numel(),
        }
    else:
        print("Unsupported module type for analysis:", type(module))


model_name = 'jsc-less-tiny'

CHECKPOINT_PATH = "/content/drive/MyDrive/AppliedMachineLearning/AdvancedDeepLearningSystems/MASErepo2/mase2/mase_output/jsc_tinyX10/software/training_ckpts/best_ep10_bs256.ckpt"
model_info = get_model_info(model_name)
model = get_model(
    model_name,
    task="cls",
    dataset_info=data_module.dataset_info,
    pretrained=False)

model = load_model(load_name=CHECKPOINT_PATH, load_type="pl", model=model)

input_generator = InputGenerator(
    data_module=data_module,
    model_info=model_info,
    task="cls",
    which_dataloader="train",
)

dummy_in = next(iter(input_generator))
_ = model(**dummy_in)

mg = MaseGraph(model=model)
mg, _ = init_metadata_analysis_pass(mg, None)
mg, _ = add_common_metadata_analysis_pass(mg, {"dummy_in": dummy_in})


from chop.passes.graph.utils import get_node_actual_target
#from chop.passes.graph.analysis.flop_estimator.calculator.calc_modules import calculate_modules # modified to account for BatchNorm1d as well

def computational_complexity_report_pass(graph, include_bias=True):
  store_data = {}
  for node in graph.fx_graph.nodes:
    try:
      in_data = (node.meta['mase'].parameters['common']['args']['data_in_0']['value'],)
    except KeyError:
      in_data = (None,)
    out_data = (node.meta['mase'].parameters['common']['results']['data_out_0']['value'],)


    module = get_node_actual_target(node)
    if isinstance(module, torch.nn.Module):
      data = calculate_modules(module, node, in_data, out_data, include_bias)

      store_data[module] = data

  count_flops = 0
  count_bitops = 0
  for key in store_data.keys():
    count_flops += store_data[key]['computations']
    count_bitops += store_data[key]['bitops']
  print(f"Total FLOPs: {count_flops}; Total BitOPs: {count_bitops}")
  return {
      "flops": count_flops,
      "bitops": count_bitops,
          }

complexity_metrics = computational_complexity_report_pass(mg, include_bias=True) # dict; keys: flops, bitops

___

# Lab 3

In [ ]:
import sys
import logging
import os
from pathlib import Path
from pprint import pprint as pp

# figure out the correct path
machop_path = Path(".").resolve().parent /"machop"
assert machop_path.exists(), "Failed to find machop at: {}".format(machop_path)
sys.path.append(str(machop_path))

from chop.dataset import MaseDataModule, get_dataset_info
from chop.tools.logger import get_logger

from chop.passes.graph.analysis import (
    report_node_meta_param_analysis_pass,
    profile_statistics_analysis_pass,
)
from chop.passes.graph import (
    add_common_metadata_analysis_pass,
    init_metadata_analysis_pass,
    add_software_metadata_analysis_pass,
)
from chop.tools.get_input import InputGenerator
from chop.ir.graph.mase_graph import MaseGraph

from chop.models import get_model_info, get_model


logger = get_logger("chop")
logger.setLevel(logging.INFO)

batch_size = 8
model_name = "jsc-less-tiny"
dataset_name = "jsc"


data_module = MaseDataModule(
    name=dataset_name,
    batch_size=batch_size,
    model_name=model_name,
    num_workers=0,
    # custom_dataset_cache_path="../../chop/dataset"
)
data_module.prepare_data()
data_module.setup()

CHECKPOINT_PATH = "/content/drive/MyDrive/AppliedMachineLearning/AdvancedDeepLearningSystems/MASErepo2/mase2/mase_output/jsc_tinyX10/software/training_ckpts/best_ep10_bs256.ckpt"
model_info = get_model_info(model_name)
model = get_model(
    model_name,
    task="cls",
    dataset_info=data_module.dataset_info,
    pretrained=False)

model = load_model(load_name=CHECKPOINT_PATH, load_type="pl", model=model)

input_generator = InputGenerator(
    data_module=data_module,
    model_info=model_info,
    task="cls",
    which_dataloader="train",
)

dummy_in = next(iter(input_generator))
_ = model(**dummy_in)

mg = MaseGraph(model=model)


# Formulate Search Space

pass_args = {
"by": "type",
"default": {"config": {"name": None}},
"linear": {
        "config": {
            "name": "integer",
            # data
            "data_in_width": 8,
            "data_in_frac_width": 4,
            # weight
            "weight_width": 8,
            "weight_frac_width": 4,
            # bias
            "bias_width": 8,
            "bias_frac_width": 4,
        }
},}

import copy
# build a search space
data_in_frac_widths = [(16, 8), (8, 6), (8, 4), (4, 2)]
w_in_frac_widths = [(16, 8), (8, 6), (8, 4), (4, 2)]
search_spaces = []
for d_config in data_in_frac_widths:
    for w_config in w_in_frac_widths:
        pass_args['linear']['config']['data_in_width'] = d_config[0]
        pass_args['linear']['config']['data_in_frac_width'] = d_config[1]
        pass_args['linear']['config']['weight_width'] = w_config[0]
        pass_args['linear']['config']['weight_frac_width'] = w_config[1]
        # dict.copy() and dict(dict) only perform shallow copies
        # in fact, only primitive data types in python are doing implicit copy when a = b happens
        search_spaces.append(copy.deepcopy(pass_args))

# Grid Search

import torch
from torchmetrics.classification import MulticlassAccuracy
import time
import numpy as np

mg, _ = init_metadata_analysis_pass(mg, None)
mg, _ = add_common_metadata_analysis_pass(mg, {"dummy_in": dummy_in})
mg, _ = add_software_metadata_analysis_pass(mg, None)

metric = MulticlassAccuracy(num_classes=5)
num_batchs = 100

def measure_throughput(model, data_loader, num_batchs):
    model.eval()
    total_inferences = 0
    start_time = time.time()
    j = 0
    for input in data_loader:
      xs, ys = inputs

      model(xs)
      total_inferences += len(xs)

      if j > num_batchs:
          break
      j += 1

    elapsed_time = time.time() - start_time
    throughput = total_inferences / elapsed_time

    return throughput

recorded_accs = []
recorded_losses = []
recorded_inference_time = []
recorded_flops = []
recorded_bitops = []
recorded_throughput = []
for i, config in enumerate(search_spaces):
  mg_quantized, _ = quantize_transform_pass(mg, config)
  j = 0

  # this is the inner loop, where we also call it as a runner.
  acc_avg, loss_avg = 0, 0
  inference_time_avg = 0
  accs, losses = [], []
  inference_time = []
  for inputs in data_module.train_dataloader(): #use test_dataloader?
      xs, ys = inputs

      starting_time = time.time()
      preds = mg_quantized.model(xs)
      inference_time.append(time.time() - starting_time)

      loss = torch.nn.functional.cross_entropy(preds, ys)
      acc = metric(preds, ys)
      accs.append(acc.item())
      losses.append(loss)
      if j > num_batchs:
          break
      j += 1

  throughput = measure_throughput(mg_quantized.model, data_module.train_dataloader(), num_batchs)
  acc_avg = sum(accs) / len(accs)
  loss_avg = sum(losses) / len(losses)
  inference_time_avg = sum(inference_time) / len(inference_time)
  recorded_accs.append(np.round(acc_avg, 2))
  recorded_losses.append(loss_avg)

  recorded_inference_time.append(np.round(inference_time_avg*1000, 2))

  recorded_throughput.append(np.round(throughput))

  complexity_metrics = computational_complexity_report_pass(mg_quantized)
  recorded_flops.append(np.round(complexity_metrics['flops']))
  recorded_bitops.append(np.round(complexity_metrics['bitops']))
  #recorded_flops.append(flops)

for i, (accuracy, inference_time, flops, bitops, throughput, search_space) in enumerate(zip(recorded_accs, recorded_inference_time, recorded_flops, recorded_bitops, recorded_throughput, search_spaces)):
  print(f"Trial {i}")
  print(f"\tAccuracy: {accuracy} | Inference Time: {inference_time} | FLOPs: {flops} | BitOPs: {bitops} | Throughput: {throughput}")
  print(f"\t{search_space['linear']['config']}")
  print("\n")

In [ ]:
# Reshape the lists into 4x4 matrices
acc_matrix = np.array(recorded_accs).reshape(4, 4)
time_matrix = np.array(recorded_inference_time).reshape(4, 4)
bitops_matrix = np.array(recorded_bitops).reshape(4, 4)
throughput_matrix = np.array(recorded_throughput).reshape(4, 4)

# Define the labels for the axes based on your configurations
data_in_labels = ['(16,8)', '(8,6)', '(8,4)', '(4,2)']
w_in_labels = ['(16,8)', '(8,6)', '(8,4)', '(4,2)']

# Set up the matplotlib figure for 4 subplots
fig, axs = plt.subplots(2, 2, figsize=(15, 12))

# Titles for each subplot
titles = ['Accuracy', 'Inference Time', 'BitOPs', 'Throughput']

# Data for each subplot
data_matrices = [acc_matrix, time_matrix, bitops_matrix, throughput_matrix]

# Customization for each heatmap
for ax, data_matrix, title in zip(axs.flat, data_matrices, titles):
    sns.heatmap(data_matrix, annot=True, fmt=".2f", cmap='Blues',
                xticklabels=data_in_labels, yticklabels=w_in_labels,
                ax=ax)
    ax.set_title(title, fontsize=16, weight='bold')
    ax.set_xlabel('Data Input Widths', fontsize=14, weight='bold')
    ax.set_ylabel('Weight Widths', fontsize=14, weight='bold')
    ax.xaxis.set_tick_params(rotation=45, labelsize=12)
    ax.yaxis.set_tick_params(rotation=45, labelsize=12)
    cbar = ax.collections[0].colorbar
    cbar.remove()

plt.tight_layout()
plt.show()

In [ ]:
# Lab 3 Question 4

!./ch search --load-type "pl" --config "configs/examples/jsc_less_tiny_by_name_search_L3Q4_TPE.toml" --load "../mase_output/jsc_tinyX10/software/training_ckpts/best_ep10_bs256.ckpt" --project-dir "/content/drive/MyDrive/AppliedMachineLearning/AdvancedDeepLearningSystems/MASErepo2/mase2/mase_output/searches_output/L3Q4"



In [ ]:
'''
import json

with open('/content/drive/MyDrive/AppliedMachineLearning/AdvancedDeepLearningSystems/MASErepo2/mase2/mase_output/searches_output/L3Q4/jsc_tinyX10/software/search_ckpts/best_results.json') as json_file:
    best_results = json.load(json_file)

print(f"Best Trial: {best_results['best_trial']} | Best Accuracy: {best_results['acc']}")
print(f"Best Configuration:")
print(best_results['config'])
'''

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/drive/MyDrive/AppliedMachineLearning/AdvancedDeepLearningSystems/MASErepo2/mase2/mase_output/searches_output/L3Q4/jsc_tinyX10/software/tensorboard/lightning_logs


___
# Lab 4

In [ ]:
import sys
import logging
import os
from pathlib import Path
from pprint import pprint as pp


# figure out the correct path
machop_path = Path(".").resolve().parent.parent /"machop"
#assert machop_path.exists(), "Failed to find machop at: {}".format(machop_path)
#sys.path.append(str(machop_path))

from chop.dataset import MaseDataModule, get_dataset_info
from chop.tools.logger import set_logging_verbosity, get_logger

from chop.passes.graph.analysis import (
    report_node_meta_param_analysis_pass,
    profile_statistics_analysis_pass,
)
from chop.passes.graph import (
    add_common_metadata_analysis_pass,
    init_metadata_analysis_pass,
    add_software_metadata_analysis_pass,
)
from chop.tools.get_input import InputGenerator
from chop.ir.graph.mase_graph import MaseGraph

from chop.models import get_model_info, get_model

set_logging_verbosity("info")

logger = get_logger("chop")
logger.setLevel(logging.INFO)

batch_size = 8
model_name = "jsc-tiny"
dataset_name = "jsc"

model_name = "jsc-tiny"

data_module = MaseDataModule(
    name=dataset_name,
    batch_size=batch_size,
    model_name=model_name,
    num_workers=0,
)
data_module.prepare_data()
data_module.setup()

model_info = get_model_info(model_name)
dataset_info = get_dataset_info(dataset_name)

input_generator = InputGenerator(
    data_module=data_module,
    model_info=model_info,
    task="cls",
    which_dataloader="train",
)

dummy_in = {"x": next(iter(data_module.train_dataloader()))[0]}

from torch import nn
from chop.passes.graph.utils import get_parent_name

# define a new model
class JSC_Three_Linear_Layers(nn.Module):
    def __init__(self):
        super(JSC_Three_Linear_Layers, self).__init__()
        self.seq_blocks = nn.Sequential(
            nn.BatchNorm1d(16),  # 0
            nn.ReLU(16),  # 1
            nn.Linear(16, 16),  # linear  2
            nn.Linear(16, 16),  # linear  3
            nn.Linear(16, 5),   # linear  4
            nn.ReLU(5),  # 5
        )

    def forward(self, x):
        return self.seq_blocks(x)

model = JSC_Three_Linear_Layers()

# generate the mase graph and initialize node metadata
mg = MaseGraph(model=model)
mg, _ = init_metadata_analysis_pass(mg, None)

def instantiate_linear(in_features, out_features, bias):
    if bias is not None:
        bias = True
    return nn.Linear(
        in_features=in_features,
        out_features=out_features,
        bias=bias)

def redefine_linear_transform_pass(graph, pass_args=None):
    main_config = pass_args.pop('config')
    default = main_config.pop('default', None)
    if default is None:
        raise ValueError(f"default value must be provided.")
    i = 0
    for node in graph.fx_graph.nodes:
        i += 1
        # if node name is not matched, it won't be tracked
        config = main_config.get(node.name, default)['config']
        name = config.get("name", None)
        if name is not None:
            ori_module = graph.modules[node.target]
            in_features = ori_module.in_features
            out_features = ori_module.out_features
            bias = ori_module.bias
            if name == "output_only":
                out_features = out_features * config["channel_multiplier"]
            elif name == "both":
                in_features = in_features * config["channel_multiplier"]
                out_features = out_features * config["channel_multiplier"]
            elif name == "input_only":
                in_features = in_features * config["channel_multiplier"]
            new_module = instantiate_linear(in_features, out_features, bias)
            parent_name, name = get_parent_name(node.target)
            setattr(graph.modules[parent_name], name, new_module)
    return graph, {}


pass_config = {
"by": "name",
"default": {"config": {"name": None}},
"seq_blocks_2": {
    "config": {
        "name": "output_only",
        # weight
        "channel_multiplier": 2,
        }
    },
"seq_blocks_3": {
    "config": {
        "name": "both",
        "channel_multiplier": 2,
        }
    },
"seq_blocks_4": {
    "config": {
        "name": "input_only",
        "channel_multiplier": 2,
        }
    },
}

# this performs the architecture transformation based on the config
# mg, _ = redefine_linear_transform_pass(
#     graph=mg, pass_args={"config": pass_config})

# define a new model
class JSC_Three_Linear_Layers(nn.Module):
    def __init__(self):
        super(JSC_Three_Linear_Layers, self).__init__()
        self.seq_blocks = nn.Sequential(
            nn.BatchNorm1d(16),  # 0
            nn.ReLU(16),  # 1
            nn.Linear(16, 16),  # linear seq_2
            nn.ReLU(16),  # 3
            nn.Linear(16, 16),  # linear seq_4
            nn.ReLU(16),  # 5
            nn.Linear(16, 5),  # linear seq_6
            nn.ReLU(5),  # 7
        )

    def forward(self, x):
        return self.seq_blocks(x)

In [ ]:
# Lab 4 Question 1


model = JSC_Three_Linear_Layers()

mg = MaseGraph(model=model)
mg, _ = init_metadata_analysis_pass(mg, None)

print("Original Graph:")
for block in mg.model.seq_blocks._modules:
  print(f"Block number {block}: {mg.model.seq_blocks._modules[block]}")


pass_config = {
"by": "name",
"default": {"config": {"name": None}},
"seq_blocks_2": {
    "config": {
        "name": "output_only",
        # weight
        "channel_multiplier": 2,
        }
    },
"seq_blocks_4": {
    "config": {
        "name": "both",
        "channel_multiplier": 2,
        }
    },
"seq_blocks_6": {
    "config": {
        "name": "input_only",
        "channel_multiplier": 2,
        }
    },
}

mg, _ = redefine_linear_transform_pass(
    graph=mg, pass_args={"config": pass_config})

print("Expanded Graph:")
for block in mg.model.seq_blocks._modules:
  print(f"Block number {block}: {mg.model.seq_blocks._modules[block]}")


In [ ]:
# Lab 4 Question 2

import torch
from torchmetrics.classification import MulticlassAccuracy
import numpy as np
import copy

from chop.actions.train import train
from chop.actions.test import test

### Define Search Space

pass_config = {
"by": "name",
"default": {"config": {"name": None}},
"seq_blocks_2": {
    "config": {
        "name": "output_only",
        "channel_multiplier": 2,
        }
    },
"seq_blocks_4": {
    "config": {
        "name": "both",
        "channel_multiplier": 2,
        }
    },
"seq_blocks_6": {
    "config": {
        "name": "input_only",
        "channel_multiplier": 2,
        }
    },
}

channel_multipliers = [1, 2, 3, 4]

# channel_multiplier_block2 = [(16, 8), (8, 6), (8, 4), (4, 2)]
# channel_multiplier_block4 = [(16, 8), (8, 6), (8, 4), (4, 2)]
# channel_multiplier_block6 = [(16, 8), (8, 6), (8, 4), (4, 2)]

# define the search space
search_spaces = []
for channel_multiplier in channel_multipliers:
  pass_config['seq_blocks_2']['config']['channel_multiplier'] = channel_multiplier
  pass_config['seq_blocks_4']['config']['channel_multiplier'] = channel_multiplier
  pass_config['seq_blocks_6']['config']['channel_multiplier'] = channel_multiplier
  search_spaces.append(copy.deepcopy(pass_config))

# define the metric to maximize in search
metric = MulticlassAccuracy(num_classes=5)
batch_size = 128

# define model, dataset, task
model_name = "jsc-three-linear-layers"
dataset_name = "jsc"
task = "cls"

model_info = get_model_info(model_name)
dataset_info = get_dataset_info(dataset_name)

data_module = MaseDataModule(
    name=dataset_name,
    batch_size=batch_size,
    model_name=model_name,
    num_workers=0,
)

data_module.prepare_data()
data_module.setup()

# trainer args required by the train function; use only the default arguments for this search (except max epochs: 20 -> 5)
plt_trainer_args = {
    "max_epochs": 5,
    "max_steps": -1,
    "devices": 1,
    "num_nodes": 1,
    "accelerator": 'gpu',
    "strategy": 'auto',
    "fast_dev_run": False,
    "precision": "16-mixed",
    "accumulate_grad_batches": 1,
    "log_every_n_steps": 50,
}


def brute_force(search_spaces):

  best_acc = 0
  best_multiplier = 1

  recorded_accs = []
  for i, config in enumerate(search_spaces):
    model = JSC_Three_Linear_Layers()
    config = copy.deepcopy(config)

    mg = MaseGraph(model=model)
    mg, _ = init_metadata_analysis_pass(mg, None)

    print("Original Graph:")
    for block in mg.model.seq_blocks._modules:
      print(f"Block number {block}: {mg.model.seq_blocks._modules[block]}")

    mg, _ = redefine_linear_transform_pass(mg, {"config": config})

    print("Expanded Graph:")
    for block in mg.model.seq_blocks._modules:
      print(f"Block number {block}: {mg.model.seq_blocks._modules[block]}")

    model = mg.model

    train(model, model_info, data_module, dataset_info, task,
          optimizer="adam", learning_rate=1e-5, weight_decay=0,
          plt_trainer_args=plt_trainer_args, auto_requeue=False,
          save_path=None, visualizer=None, load_name=None, load_type=None)

    test_results = test(model, model_info, data_module, dataset_info, task,
                        optimizer="adam", learning_rate=1e-5, weight_decay=0,
                        plt_trainer_args=plt_trainer_args, auto_requeue=False,
                        save_path=None, visualizer=None, load_name=None, load_type=None,
                      return_test_results=True)

    acc_avg = test_results[0]['test_acc_epoch']
    loss_avg = test_results[0]['test_loss_epoch']
    recorded_accs.append(acc_avg)

    if acc_avg > best_acc:
      best_acc = acc_avg
      best_multiplier = config['seq_blocks_2']['config']['channel_multiplier']

  print(f"Best Accuracy: {best_acc}\nBest Channel Multiplier: {best_multiplier}")

  return best_acc, best_multiplier, recorded_accs


best_acc, best_multiplier, recorded_accs = brute_force(search_spaces)





In [ ]:
# Lab 4 Question 3

import torch
from torchmetrics.classification import MulticlassAccuracy
import numpy as np
import copy

from chop.actions.train import train
from chop.actions.test import test

### Define Search Space

pass_config = {
"by": "name",
"default": {"config": {"name": None}},
"seq_blocks_2": {
    "config": {
        "name": "output_only",
        "channel_multiplier": 2,
        }
    },
"seq_blocks_4": {
    "config": {
        "name": "both",
        "channel_multiplier_in": 2,
        "channel_multiplier_out": 2,
        }
    },
"seq_blocks_6": {
    "config": {
        "name": "input_only",
        "channel_multiplier": 2,
        }
    },
}

channel_multipliers = [1, 2, 3, 4]

search_spaces = []
for channel_multiplier_1 in channel_multipliers:
  for channel_multiplier_2 in channel_multipliers:
    pass_config['seq_blocks_2']['config']['channel_multiplier'] = channel_multiplier_1
    pass_config['seq_blocks_4']['config']['channel_multiplier_in'] = channel_multiplier_1
    pass_config['seq_blocks_4']['config']['channel_multiplier_out'] = channel_multiplier_2
    pass_config['seq_blocks_6']['config']['channel_multiplier'] = channel_multiplier_2
    search_spaces.append(copy.deepcopy(pass_config))

def redefine_linear_transform_pass(graph, pass_args=None):
    main_config = pass_args.pop('config')
    default = main_config.pop('default', None)
    if default is None:
        raise ValueError(f"default value must be provided.")
    i = 0
    for node in graph.fx_graph.nodes:
        i += 1
        # if node name is not matched, it won't be tracked
        config = main_config.get(node.name, default)['config']
        name = config.get("name", None)
        if name is not None:
            ori_module = graph.modules[node.target]
            in_features = ori_module.in_features
            out_features = ori_module.out_features
            bias = ori_module.bias
            if name == "output_only":
                out_features = out_features * config["channel_multiplier"]
            elif name == "both":
                in_features = in_features * config["channel_multiplier_in"]
                out_features = out_features * config["channel_multiplier_out"]
            elif name == "input_only":
                in_features = in_features * config["channel_multiplier"]
            new_module = instantiate_linear(in_features, out_features, bias)
            parent_name, name = get_parent_name(node.target)
            setattr(graph.modules[parent_name], name, new_module)
    return graph, {}

def brute_force(search_spaces):

  best_acc = 0

  recorded_accs = []

  study = {}

  for i, config in enumerate(search_spaces):
    model = JSC_Three_Linear_Layers()
    config = copy.deepcopy(config)

    mg = MaseGraph(model=model)
    mg, _ = init_metadata_analysis_pass(mg, None)

    print("Original Graph:")
    for block in mg.model.seq_blocks._modules:
      print(f"Block number {block}: {mg.model.seq_blocks._modules[block]}")

    mg, _ = redefine_linear_transform_pass(mg, {"config": config})

    print("Expanded Graph:")
    for block in mg.model.seq_blocks._modules:
      print(f"Block number {block}: {mg.model.seq_blocks._modules[block]}")

    model = mg.model

    train(model, model_info, data_module, dataset_info, task,
          optimizer="adam", learning_rate=1e-5, weight_decay=0,
          plt_trainer_args=plt_trainer_args, auto_requeue=False,
          save_path=None, visualizer=None, load_name=None, load_type=None)

    test_results = test(model, model_info, data_module, dataset_info, task,
                        optimizer="adam", learning_rate=1e-5, weight_decay=0,
                        plt_trainer_args=plt_trainer_args, auto_requeue=False,
                        save_path=None, visualizer=None, load_name=None, load_type=None,
                      return_test_results=True)

    acc_avg = test_results[0]['test_acc_epoch']
    loss_avg = test_results[0]['test_loss_epoch']
    recorded_accs.append(acc_avg)

    if acc_avg > best_acc:
      best_acc = acc_avg
      best_multiplier_1 = config['seq_blocks_2']['config']['channel_multiplier']
      best_multiplier_2 = config['seq_blocks_6']['config']['channel_multiplier']

  print(f"Best Accuracy: {best_acc}\nBest Channel Multiplier 1: {best_multiplier_1}\nBest Channel Multiplier 2: {best_multiplier_2}")

  return best_acc, best_multiplier_1, best_multiplier_2, recorded_accs


best_acc, best_multiplier_1, best_multiplier_2, recorded_accs = brute_force(search_spaces)





In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

acc_matrix = np.array(recorded_accs).reshape(4, 4)

channel_multiplier_1 = ['1', '2', '3', '4']
channel_multiplier_2 = ['1', '2', '3', '4']

# Create the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(acc_matrix, annot=True, fmt=".2f", cmap='Blues',
            xticklabels=channel_multiplier_1, yticklabels=channel_multiplier_2)

plt.xlabel('Channel Multiplier 1')
plt.ylabel('Channel Multiplier 2')
plt.show()

In [ ]:
# Lab 4 Question 4

!./ch search --load-type "pl" --config "configs/examples/search_linear_channel_multiplier.toml" --project-dir "/content/drive/MyDrive/AppliedMachineLearning/AdvancedDeepLearningSystems/MASErepo2/mase2/mase_output/searches_output/L4Q4"



In [ ]:
# Lab 4 - Optional Question

!./ch search --load-type "pl" --config "configs/examples/search_convolutional_filters_multiplier.toml" --project-dir "/content/drive/MyDrive/AppliedMachineLearning/AdvancedDeepLearningSystems/MASErepo2/mase2/mase_output/searches_output/L4Q5"

